In [1]:
import pandas as pd
import numpy as np

import time

from datasource import * 

[1] vnpy合成日k线周k线: https://blog.csdn.net/weixin_43769878/article/details/118493832#comments_22703925

In [2]:
def get_new_kline(source_df:pd.DataFrame, target_interval = '5S'):
    # 假设 source_df 包含 'datetime', 'price', 'volume' 列
    # 设置 'datetime' 列为 DataFrame 的索引
    source_df.set_index('datetime', inplace=True)

    # 使用 resample 合成5秒K线
    new_kline_df = source_df.resample(target_interval).agg({
        'price': 'ohlc',   # open, high, low, close
        'volume': 'sum'
    })

    # 合成1秒K线
    # kline_data = pd.DataFrame({
    #     'open': source_df['price'].resample('1S').first(),
    #     'high': source_df['price'].resample('1S').max(),
    #     'low': source_df['price'].resample('1S').min(),
    #     'close': source_df['price'].resample('1S').last(),
    #     'volume': source_df['volume'].resample('1S').sum(),
    # })

    # 重置索引，将 'datetime' 列还原为普通列
    new_kline_df.reset_index(inplace=True)

    # 为了符合你的要求，可能需要处理一些缺失值等情况
    # 如果需要，可以添加进一步的处理逻辑

    # 打印结果
    print(new_kline_df)
    return new_kline_df

In [3]:
def get_new_kline_v1(source_df, target_interval):
    ret = []

    if source_df is None or len(source_df) <= 0:
        return None

    source_len = len(source_df)

    is_begin = False
    count = 0
    open = 0
    close = 0
    high = 0
    low = 0
    volume = 0

    for i in range(1, source_len):  # Start from index 1 to calculate diff
        time_diff = (source_df.iloc[i]['datetime'] - source_df.iloc[0]['datetime']).total_seconds() * 1000

        if (time_diff % target_interval == 0) or (source_df.iloc[i]['datetime'].microsecond == 0):
            is_begin = True

        if is_begin:
            if count == 0:
                datetime = source_df.iloc[i]['datetime']
                open = source_df.iloc[i]['price']
                close = source_df.iloc[i]['price']
                high = source_df.iloc[i]['price']
                low = source_df.iloc[i]['price']
                volume = source_df.iloc[i]['volume']
                count += 1
            elif count < (int(time_diff / target_interval)):
                close = source_df.iloc[i]['price']
                high = max(high, source_df.iloc[i]['price'])
                low = min(low, source_df.iloc[i]['price'])
                volume += source_df.iloc[i]['volume']
                count += 1

            if count == (int(time_diff / target_interval)) or i == source_len - 1:
                ret.append({
                    'datetime': datetime,
                    'open': open,
                    'close': close,
                    'high': high,
                    'low': low,
                    'volume': volume,
                })
                count = 0

    columns = [
        "datetime",
        "open",
        "close",
        "high",
        "low",
        "volume"
    ]
    return pd.DataFrame(ret, columns=columns)


In [4]:
code_list = [
    ['518880.XSHG','518880.XSHG'], #黄金ETF  518880
    ['513100.XSHG','513100.XSHG'], #纳指ETF  513100
    ['399006.XSHE','159915.XSHE'], #创业板 399006 159915 159952 512900
    # ['513050.XSHG','513050.XSHG'], #中概互联 ETF  
    # ['399673.XSHE','399673.XSHE'], #创业板50 399673 159949
    ['000016.XSHG','000016.XSHG'], #上证50 000016 510050 510710 510850
    # ['000300.XSHG','510300.XSHG'], #沪深300
    # ['000905.XSHG','510500.XSHG'] #中证500
    # ['399975.XSHE','399975.XSHE'], #证劵公司 399975 512880 512000 512900
    # ['399987.XSHE','399987.XSHE'], #中证酒
    # ['399932.XSHE','159928.XSHE'], #消费ETF
    # ['000913.XSHG','000913.XSHG'], #300医药 000913 399913 000913
    # ['515000.XSHG','515000.XSHG'], #科技 515000
    # ['000015.XSHG','510880.XSHG'], #红利ETF
]

code_list_ = [l[1].split(".")[0] for l in code_list]

def get_source_df():
    fund_etf_hist_min_em_df = fund_etf_hist_min_em(
        symbol = code_list_[2],
        period="1",
        adjust="qfq",
        start_date="2024-01-02 09:30:00",
        end_date="2024-01-08 15:00:00",
    )

    fund_etf_hist_min_em_df.columns = [
        "date",
        "open",
        "close",
        "high",
        "low",
        "volume",
        "amount",
        "price",
    ]

    source_df = fund_etf_hist_min_em_df
    source_df["date"] = pd.to_datetime(source_df["date"])
    # source_df = fund_etf_hist_min_em_df.set_index(["date"], drop=True, inplace=False)
    print(source_df)
    return source_df

source_df = get_source_df()

                   date  open  close   high    low  volume      amount   price
0   2024-01-03 09:30:00   0.0  1.802  1.802  1.802   94990  17117198.0  1.8020
1   2024-01-03 09:31:00   0.0  1.800  1.803  1.800  152645  27492771.0  1.8014
2   2024-01-03 09:32:00   0.0  1.802  1.803  1.800   85451  15386569.0  1.8012
3   2024-01-03 09:33:00   0.0  1.805  1.806  1.803  146436  26423795.0  1.8022
4   2024-01-03 09:34:00   0.0  1.807  1.807  1.804   50899   9193101.0  1.8026
..                  ...   ...    ...    ...    ...     ...         ...     ...
959 2024-01-08 14:56:00   0.0  1.699  1.700  1.699  126421  21482267.0  1.7130
960 2024-01-08 14:57:00   0.0  1.701  1.701  1.698  193750  32921698.0  1.7128
961 2024-01-08 14:58:00   0.0  1.700  1.700  1.700     573     97410.0  1.7128
962 2024-01-08 14:59:00   0.0  1.700  1.700  1.700       0         0.0  1.7128
963 2024-01-08 15:00:00   0.0  1.699  1.699  1.699  129431  21990326.0  1.7127

[964 rows x 8 columns]


In [7]:
def tick_data_mock():
    # 设置交易时间范围
    trading_hours_1 = pd.date_range("2024-01-01 09:30:00", "2024-01-01 11:30:00", freq='S')
    trading_hours_2 = pd.date_range("2024-01-01 13:00:00", "2024-01-01 15:00:00", freq='S')
    trading_hours = trading_hours_1.union(trading_hours_2)

    # 生成tick数据
    tick_data = pd.DataFrame({
        'datetime': trading_hours,
        'price': np.random.rand(len(trading_hours)) * 10 + 100,  # 生成随机价格，假设在100到110之间
        'volume': np.random.randint(1, 100, size=len(trading_hours))  # 生成随机成交量
    })

    # 打印生成的tick数据
    print(tick_data)
    return tick_data

In [8]:
tick_data = tick_data_mock()

                 datetime       price  volume
0     2024-01-01 09:30:00  102.972389      37
1     2024-01-01 09:30:01  105.014955      60
2     2024-01-01 09:30:02  104.504523      84
3     2024-01-01 09:30:03  105.948319      31
4     2024-01-01 09:30:04  103.050005      93
...                   ...         ...     ...
14397 2024-01-01 14:59:56  102.400742      61
14398 2024-01-01 14:59:57  108.928681      90
14399 2024-01-01 14:59:58  102.616866      29
14400 2024-01-01 14:59:59  106.792101      36
14401 2024-01-01 15:00:00  103.147806       5

[14402 rows x 3 columns]


In [9]:
new_1_df = get_new_kline(tick_data)

                datetime       price                                      \
                                open        high         low       close   
0    2024-01-01 09:30:00  102.972389  105.948319  102.972389  103.050005   
1    2024-01-01 09:30:05  109.947147  109.947147  103.461426  109.823622   
2    2024-01-01 09:30:10  104.274582  109.718613  100.170927  107.447022   
3    2024-01-01 09:30:15  106.336171  109.682462  106.336171  107.304654   
4    2024-01-01 09:30:20  103.173693  109.461434  100.230431  103.942326   
...                  ...         ...         ...         ...         ...   
3956 2024-01-01 14:59:40  101.018117  109.416168  101.018117  104.354973   
3957 2024-01-01 14:59:45  103.717044  107.996919  100.994891  104.363536   
3958 2024-01-01 14:59:50  108.473414  109.299613  102.585748  103.757040   
3959 2024-01-01 14:59:55  106.676803  108.928681  102.400742  106.792101   
3960 2024-01-01 15:00:00  103.147806  103.147806  103.147806  103.147806   

     volume

In [11]:
from data_repo_v1 import get_new_cycle_records

new_1_df.columns = [
    "date",
    "open",
    "high",
    "low",      
    "close",
    "volume",
]

new_1m_df = get_new_cycle_records(new_1_df, target_cycle=1000*60)

In [12]:
new_1m_df

,date,open,close,high,low,volume
0,2024-01-01 09:30:00,102.972389,101.477420,109.947147,100.170927,2878
1,2024-01-01 09:31:00,107.946533,103.315995,109.680800,100.010188,2979
2,2024-01-01 09:32:00,102.020633,105.688918,109.925172,100.035045,3237
3,2024-01-01 09:33:00,109.876163,101.942385,109.993592,100.047748,2819
4,2024-01-01 09:34:00,108.881693,100.869924,109.939650,100.619781,3373
...,...,...,...,...,...,...
326,2024-01-01 14:56:00,107.513995,101.385229,109.499423,100.189997,2795
327,2024-01-01 14:57:00,102.626728,102.307316,109.886331,100.170714,2789
328,2024-01-01 14:58:00,101.732740,102.606675,109.840352,100.329745,3018
329,2024-01-01 14:59:00,101.004612,106.792101,109.866450,100.090740,2822


In [13]:
new_15s_df = get_new_cycle_records(new_1_df, target_cycle=1000*15)

In [14]:
new_15s_df

,date,open,close,high,low,volume
0,2024-01-01 09:30:00,102.972389,107.447022,109.947147,100.170927,775
1,2024-01-01 09:30:15,106.336171,103.838233,109.682462,100.230431,817
2,2024-01-01 09:30:30,100.456643,100.714489,109.863953,100.456643,737
3,2024-01-01 09:30:45,107.711253,101.477420,109.337103,100.915094,549
4,2024-01-01 09:31:00,107.946533,102.238829,108.754254,100.010188,796
...,...,...,...,...,...,...
1316,2024-01-01 14:59:00,101.004612,108.563522,109.445120,101.004612,655
1317,2024-01-01 14:59:15,107.837860,107.135921,109.866450,100.991550,842
1318,2024-01-01 14:59:30,108.905355,104.354973,109.835240,100.090740,637
1319,2024-01-01 14:59:45,103.717044,106.792101,109.299613,100.994891,688
